动态规划是20世纪50年代由Richard Bellman发明的。不像贪婪算法，回溯算法等，单从名字上根本理解不了这是什么鬼。Bellman自己也说了，这个名字完全是为了申请经费搞出来的（囧），所以说这个名字坑了一代又一代的人啊。

言归正传，我们来了解下动态规划，dynamic Programming，是一种高效解决问题的方法，使用与具有重复子问题和最优子结构的问题。（又是两个搞不懂的名词啊）。不过没问题，我们可以通过举例子来说明。

如果可以把局部子问题的解结合起来得到全局最优解，那这个问题就具备最优子结构
如果计算最优解时需要处理很多相同的问题，那么这个问题就具备重复子问题

就像看上了一个女孩，不能直接上去泡人家，要先制造一次偶遇一样，这里我们先从一个简单的问题来认识动态规划。  

Fibonacci sequence  
fibonacci数列是递归算法的一个典型的例子，这里不介绍了，大家都懂，直接上代码：


In [2]:
def Fibnacci(n):
    if n==0 or n==1:
        return 1
    else:
        return Fibnacci(n-1) + Fibnacci(n-2)
%time Fibnacci(37)

Wall time: 20.4 s


39088169

这速度也是醉了啊，这才是37位啊，手算也就2分钟的事啊。
如果试下Fibnacci(120) ，这个千万不敢试，会怀孕，说错了，是会看不到明天的太阳。（官方统计算完基本上要250000年）

那么为什么会这么慢呢。我们以Fibnacci(5) 位例子，每次计算Fibnacci(5) 都要计算Fibnacci(4) 和Fibnacci(3) ，而Fibnacci(4)要计算Fibnacci(3) 和Fibnacci(2) ，Fibnacci(3)要计算Fibnacci(2) 和Fibnacci(1) ，一直这样递归下去，作了大量的重复计算。而函数调用时很费时间和空间的。

有一个图很好的说明了这个问题：
![](./图片/斐波那契数列的递归实现.png)

既然重复计算如此耗时，那么能不能不重复计算这些值呢？当第一次计算了这些值的时候，我们把他们缓存起来，等到再次使用的时候，直接把他们拿过来用，这样就不用做大量的重复计算了。这就是动态规划的核心思想。

还是以Fibnacci为例子：
每当我们第一次计算Fibnacci(n)的时候，我们就将其缓存到memo的列表中，当需要再次计算Fibnacci(n)的时候，首先去memo的列表中查找，如果找到了就直接拿来用，找不到再计算。下面是具体的程序：


In [3]:
def fastFib(n,memo={}):
    if n==0 or n==1:
        return 1
    try:
        return memo[n]
    except KeyError:
        result = fastFib(n-1,memo) + fastFib(n-2,memo)
        memo[n] = result
        return result


In [4]:
%time fastFib(37)

Wall time: 0 ns


39088169

这就是差距啊！从算法复杂度上来讲这次的算法每次只调用fastFib函数一次，所以复杂度为O(n)
这就是差距的原因。

下一章节将是如何利用动态规划来解决0/1背包问题

# 动态规划解决0/1背包问题

之前总结了利用穷举法，贪婪法解决0/1背包的方法，同时也通过Fibnacci介绍了动态规划，那么该如何来利用动态规划来解决0/1背包问题呢？

首先动态规划有两个条件;  
如果可以把局部子问题的解结合起来得到全局最优解，那这个问题就具备最优子结构  
如果计算最优解时需要处理很多相同的问题，那么这个问题就具备重复子问题  

从这两点看，0/1背包问题跟动态规划没有半毛钱的关系啊。那这两者又是怎么联系起来的呢？我们通过二叉树将两者联系起来。  

二叉树是一种树，每个跟节点至多有两个子节点。  

我们可以将0/1背包问题通过二叉树来表示：

[0-1背包问题]有一个背包，背包容量是M=150kg。有7个物品，物品不可以分割成任意大小。（这句很重要）  
要求尽可能让装入背包中的物品总价值最大，但不能超过总容量。  
物品 A B C D E F G  
重量 35kg 30kg 6kg 50kg 40kg 10kg 25kg  
价值 10 40 30 50 35 40 30  

我们可以用下面的二叉树来表示问题的所有的解空间。
 ![](./图片/动态算法的背包问题解决.png)
 
 

借个图，实在是不想画了，见谅  

每个节点由四部分组成：  

第一个集合表示：已经拿到背包里的物品  
第二个集合表示：还没有决定要拿走的物品  
第三个值表示：当前背包里的物品总价值  
第四个值表示：背包剩余的重量  

我们按照如下的策略进行生长  

左子树表示：拿到了第二个集合中的第一个物品，右子树表示放弃掉第二个集合中的第一个物品  

那么由着这个树一直生长下去，我们可以得到最终问题的解空间。  

很明显这是一个可以用递归解决的问题。  

那么下面就首先用递归的算法先来解决这个问题  

对于递归来说要有一个边界条件，那么这里的边界条件有两个，一个是第二个集合为空（意味着全部拿走），另一个是第四个值为0（意味着背包已经装满了），而他们就是叶子节点，因为树的遍历或者说是递归只能是到达叶子节点就结束了。

普通递归方法求解：  
 

In [12]:
# item have three attribute: name,weight,value
class Item(object):
    def __init__(self,n,v,w):
        self.name=n
        self.weight=float(w)
        self.value=float(v)

    def getName(self):
        return self.name

    def getValue(self):
        return self.value

    def getWeight(self):
        return self.weight

    def __str__(self):
        result = ' < '+self.name+' , '+str(self.value)+' , '+str(self.weight) + '>'
        return result



def maxValue(oraSet,leftRoom):  
    """
    oraSet : 集合
    leftRoom : 还余下多少重量"""
    # leaf 
    if oraSet == [] or leftRoom == 0:
        return (0,())
    # only right tree
    elif oraSet[0].getWeight() > leftRoom:
        result = maxValue(oraSet[1:],leftRoom)
    # select the best from the left and right
    else:
        # left tree, means we select nextItem(the first value of the remains)
        nextItem = oraSet[0]
        leftVal, leftToken = maxValue(oraSet[1:], leftRoom - nextItem.getWeight())
        leftVal +=nextItem.getValue()

        # right tree,means we do not select nextItem
        rightVal,rightToken = maxValue(oraSet[1:],leftRoom)

        if leftVal > rightVal:
            result = (leftVal,leftToken+(nextItem,))
        else:
            result = (rightVal,rightToken)

    return result


def buildItem():
    names=['A','B','C','D','E','F','G']
    vals = [35,30,6,50,40,10,25]
    weights=[10,40,30,50,35,40,30]
    Items=[]
    for i in range(len(names)):
        Items.append(Item(names[i],vals[i],weights[i]))
    return Items

def testCode():
    value,token = maxValue(buildItem(), 150)
    for item in token:
        print(item)
    print ("Total value of tokens is {}".format( value))

%time testCode()


 < E , 40.0 , 35.0>
 < D , 50.0 , 50.0>
 < B , 30.0 , 40.0>
 < A , 35.0 , 10.0>
Total value of tokens is 155.0
Wall time: 0 ns


动态规划解法：  
要想用动态规划，首先要满足两个条件：**重复子问题 和 最优子结构**  
每个父节点会组合子节点的解来得到这个父节点为跟的子树的最优解，所以存在最优子结构。  
同一层的每个节点剩余的可选物品集合都是一样的，所以具有重复子问题  
因此可以利用动态规划来解决问题。  

动态规划的核心就是提供了一个memory，能够缓存已经计算过的值
 

In [11]:
# item have three attribute: name,weight,value
import time 
class Item(object):
    def __init__(self,n,v,w):
        self.name=n
        self.weight=float(w)
        self.value=float(v)

    def getName(self):
        return self.name

    def getValue(self):
        return self.value

    def getWeight(self):
        return self.weight

    def __str__(self):
        result = ' < '+self.name+' , '+str(self.value)+' , '+str(self.weight) + '>'
        return result


def fastMaxVal(oraSet,leftRoom,memo={}):
    if (len(oraSet),leftRoom) in memo:
        result = memo[(len(oraSet),leftRoom)]
    elif oraSet == [] or leftRoom == 0:
        result = (0,())
    elif oraSet[0].getWeight()>leftRoom:
        result = fastMaxVal(oraSet[1:],leftRoom,memo)
    else:
        nextItem = oraSet[0]

        leftValue,leftToken = fastMaxVal(oraSet[1:],leftRoom - nextItem.getWeight(),memo)
        leftValue +=nextItem.getValue()

        rightValue,rightToken = fastMaxVal(oraSet[1:],leftRoom,memo)

        if leftValue >rightValue:
            result = (leftValue,leftToken+(nextItem,))
        else:
            result = (rightValue,rightToken)

    memo[(len(oraSet),leftRoom)] = result

    return result



def buildItem():
    names=['A','B','C','D','E','F','G']
    vals = [35,30,6,50,40,10,25]
    weights=[10,40,30,50,35,40,30]
    Items=[]
    for i in range(len(names)):
        Items.append(Item(names[i],vals[i],weights[i]))
    return Items

def testCode():
    value,token = fastMaxVal(buildItem(), 150)
    for item in token:
        print(item)
    print ("Total value of tokens is ()".format( value))
%time testCode()

 < E , 40.0 , 35.0>
 < D , 50.0 , 50.0>
 < B , 30.0 , 40.0>
 < A , 35.0 , 10.0>
Total value of tokens is ()
Wall time: 7.99 ms


思考：  
1） 动态规划为什么会快？
因为这里不需要调用函数计算重复子问题，那么一定就是快很多么？不一定，这取决于重复子问题的多少。0/1背包问题当数据量大时，他的时间节省比较多的原因在与我们设计的重复子问题比较好，因为对于物品的多种组合来说，他们的剩余空间的一致的概率比较大，多以告知重复子问题会比较多。

2）动态规划的核心：
核心在于memory的设计，这里我们利用了memo[(len(oraSet),leftRoom)]中的(len(oraSet),leftRoom)字典作为键，为什么可以利用len(oraSet)而不是oraSet呢（当然oraSet也是可以的）？这是因为对于每一层的子节点来说，剩余物品的个数都是一致的，这个个数可以区分重复子问题。而动态规划相较于普通的递归算法，主要的就是增加了memory

3）如何设计动态规划算法：
首先看问题是否满足动态规划的两个条件：重复子问题，最优子结构；然后首先利用递归算法解决问题，设计memory，然后修改递归算法的实现，加入memory，最终实现动态规划的算法。
 